Libraries

In [1]:
import os
import cv2
import numpy as np
import time
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from PIL import Image
from sklearn.metrics import classification_report
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer

Constants

In [2]:
IMAGE_SIZE = 64  

Pre Processing

In [3]:
execution_start_time = time.time()
data = []
labels = []

# Function to prepare the dataset
def prepareDataset(path):
    filePaths = []
    y = []
    count = 0
    name = ''

    for dirname, _, filenames in os.walk(path):
        if count != 0:
            x = dirname.split('/')[-1]
            index = x.rindex('_name_')
            name = x[index+6:].replace(' ','')
        for filename in filenames:
            full_path = os.path.join(dirname, filename)
            filePaths.append(full_path)
            y.append(name)
        
        count += 1
    return filePaths, y

hispanic_path = 'ID_Images_Dataset/Selfies_ID_Images_dataset/11_sets_Hispanics'  
caucasian_path = 'ID_Images_Dataset/Selfies_ID_Images_dataset/18_sets_Caucasians'  

hispanic_filePaths, hispanic_labels = prepareDataset(hispanic_path)
caucasian_filePaths, caucasian_labels = prepareDataset(caucasian_path)


filePaths = hispanic_filePaths + caucasian_filePaths
y_labels = hispanic_labels + caucasian_labels  

# Load and preprocess the images
for i, filePath in enumerate(filePaths):
    if not filePath.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"Skipping non-image file: {filePath}")
        continue
    try:
        pil_image = Image.open(filePath).convert('RGB')
        image = np.array(pil_image)
        image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        data.append(image)
        labels.append(y_labels[i])
    except (IOError, OSError) as e:
        print(f"Warning: Could not read image from {filePath} - {e}")
#
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# Convert the labels from strings to integers
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# Define the number of classes for the CNN model
number_of_classes = labels.shape[1]
print("Classes Count: ",number_of_classes)
np.save('label_classes.npy', lb.classes_)

Skipping non-image file: ID_Images_Dataset/Selfies_ID_Images_dataset/11_sets_Hispanics/.DS_Store
Skipping non-image file: ID_Images_Dataset/Selfies_ID_Images_dataset/18_sets_Caucasians/.DS_Store
Classes Count:  29


Model

In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(number_of_classes, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4), 
              metrics=['accuracy'])

start_time = time.time()
model.fit(data, labels, epochs=5, batch_size=32)
training_time = time.time() - start_time
print(f"Total training time: {training_time:.2f} seconds")
model.save('face_recognition_model.h5')

Epoch 1/5
14/14 [==============================] - 1s 50ms/step - loss: 3.4287 - accuracy: 0.0368
Epoch 2/5
14/14 [==============================] - 1s 56ms/step - loss: 3.3685 - accuracy: 0.0552
Epoch 3/5
14/14 [==============================] - 1s 53ms/step - loss: 3.3320 - accuracy: 0.0667
Epoch 4/5
14/14 [==============================] - 1s 54ms/step - loss: 3.2898 - accuracy: 0.0598
Epoch 5/5
14/14 [==============================] - 1s 50ms/step - loss: 3.2619 - accuracy: 0.0874
Total training time: 4.09 seconds


/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Classification Report

In [5]:
model = load_model('face_recognition_model.h5')
predictions = model.predict(data)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(labels, axis=1)
lb.classes_ = np.load('label_classes.npy')
report = classification_report(true_classes, predicted_classes, target_names=lb.classes_)
print(report)
end_execution_time = time.time() - execution_start_time
print(f"Total Execution Time: {end_execution_time} seconds")

14/14 [==============================] - 0s 14ms/step
                 precision    recall  f1-score   support

      Alejandra       0.86      0.40      0.55        15
     Alessandro       0.86      0.80      0.83        15
      Anastasia       0.00      0.00      0.00        15
      AndreaRan       1.00      0.13      0.24        15
           Anna       0.15      0.53      0.24        15
          Bruno       0.00      0.00      0.00        15
       Clarissa       0.56      0.33      0.42        15
         Daiane       0.33      0.20      0.25        15
          Diego       0.15      0.13      0.14        15
            Ewa       0.00      0.00      0.00        15
       Fernanda       0.50      0.27      0.35        15
        Gabriel       1.00      0.20      0.33        15
GültenÇayırcı       0.27      0.20      0.23        15
        Juliana       0.00      0.00      0.00        15
          Kasia       0.29      0.13      0.18        15
       Kateryna       0.00      0

/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
